In [1]:
import os
import numpy as np
from matplotlib.lines import Line2D
from scipy.stats import pearsonr, gaussian_kde, linregress, ttest_ind, sem, zscore
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import norm
from scipy.stats import percentileofscore
from sklearn.utils.validation import check_random_state
from math import factorial
from more_itertools import distinct_permutations
import statsmodels.api as sm

import matplotlib.pyplot as plt

import pandas as pd
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

import numpy as np
from scipy.stats import linregress
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, ParameterGrid, train_test_split
from tqdm.notebook import tqdm

import warnings
import random
#from torch.utils.data import SubsetRandomSampler
from sklearn.utils import resample

from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score
from collections import Counter

from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score

warnings.filterwarnings("ignore")



In [2]:
from scipy.stats import chi2

def summarize_feature(df, feature, df_out, stat_cols=(10, 50), df_offset=2):

    d = df[df["Feature"] == feature]
    if d.empty:
        return df_out  

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            combined_p = 1 - chi2.cdf(d[col].mean(), df_chi)
            df_out.loc[feature, f"hl_pvalue_combined_{k}"] = combined_p

    # --- RR summary ---
    if "RR" in d.columns:
        or_mean = d["RR"].mean()
        or_std = d["RR"].std(ddof=1)
        df_out.loc[feature, "RR"] = or_mean
        df_out.loc[feature, "2.5%"] = or_mean - or_std
        df_out.loc[feature, "97.5%"] = or_mean + or_std

    # --- min/max de los IC originales (tal como lo estabas haciendo) ---
    if "2.5%" in d.columns:
        df_out.loc[feature, "minRR"] = d["2.5%"].min()
    if "97.5%" in d.columns:
        df_out.loc[feature, "maxRR"] = d["97.5%"].max()

    return df_out


features = ["Mono", "One", "Two", "Three", "Total"] 




def hosmer_lemeshow(y_true, y_prob, g):
    hl_df = pd.DataFrame({
        "observed": y_true,
        "predicted_probability": y_prob
    }).dropna()


    hl_df["group"] = pd.qcut(hl_df["predicted_probability"], g, duplicates="drop")

    hl_table = hl_df.groupby("group").apply(
        lambda x: pd.Series({
            "observed": x["observed"].sum(),
            "expected": x["predicted_probability"].sum(),
            "total": len(x)
        })
    )

    hl_table["observed_neg"] = hl_table["total"] - hl_table["observed"]
    hl_table["expected_neg"] = hl_table["total"] - hl_table["expected"]


    hl_statistic = (
        ((hl_table["observed"] - hl_table["expected"])**2) / hl_table["expected"] +
        ((hl_table["observed_neg"] - hl_table["expected_neg"])**2) / hl_table["expected_neg"]
    ).sum()


    dof = hl_table.shape[0] - 2
    p_value = 1 - chi2.cdf(hl_statistic, dof)

    return hl_statistic, p_value



def summarize_feature_by_covar(df, covar, feature, df_out, stat_cols=(10, 50), df_offset=2):
    d = df[(df["Covar"] == covar) & (df["Feature"] == feature)]
    if d.empty:
        return df_out

    idx = (covar, feature)

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            df_out.loc[idx, f"hl_pvalue_combined_{k}"] = 1 - chi2.cdf(d[col].mean(), df_chi)

    or_mean = d["RR"].mean()
    or_std  = d["RR"].std(ddof=1)
    df_out.loc[idx, "RR"] = or_mean
    df_out.loc[idx, "2.5%"] = or_mean - or_std
    df_out.loc[idx, "97.5%"] = or_mean + or_std

    df_out.loc[idx, "minRR"] = d["2.5%"].min()
    df_out.loc[idx, "maxRR"] = d["97.5%"].max()

    df_out.loc[idx, "n_iter"] = d.shape[0]

    return df_out


## Load data

In [3]:
import time

t0 = time.perf_counter()






In [4]:
data = pd.read_csv('data/BBAG-long.csv')

In [5]:
Counter(data.country)

Counter({'Austria': 4320,
         'Belgium': 4942,
         'Czechia': 5575,
         'Switzerland': 2634,
         'Germany': 4567,
         'Denmark': 2609,
         'Estonia': 7522,
         'Spain': 4912,
         'France': 3640,
         'Hungary': 3010,
         'Italy': 4541,
         'Netherlands': 2546,
         'Poland': 3612,
         'Portugal': 1914,
         'Sweden': 2792,
         'Slovenia': 5291,
         'Luxembourg': 2104,
         'Greece': 2599,
         'Croatia': 2798,
         'Bulgaria': 1953,
         'Cyprus': 1205,
         'Finland': 1982,
         'Lithuania': 2033,
         'Latvia': 1684,
         'Malta': 1259,
         'Romania': 2083,
         'Slovakia': 2022})

In [6]:
vars_ = ['Mono', 'One',	'Two',	'Three', 'Total']


# Relative Risk

## Without co-vars

### Without iterations

In [8]:
results_merge_df_all = data.copy()

results_merge_df_all = results_merge_df_all.loc[:, ~results_merge_df_all.columns.duplicated()]

df_directions_odd = pd.DataFrame()

for i in vars_:
    
    y_ols = results_merge_df_all['GAP_bin']

    
    X_ols = results_merge_df_all[[i] + ['delta_time']]
    X_train, X_test, y_train, y_test = train_test_split(X_ols, y_ols, test_size=0.2, random_state=42)
    

    scaler = MinMaxScaler((0.05, 0.95))
    X_train_scaled = scaler.fit_transform(X_train)
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
    
    X_test_scaled = scaler.transform(X_test)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
    
    X_train_scaled['intercept'] = 1
    X_test_scaled['intercept'] = 1
    
    model = sm.GLM(y_train, X_train_scaled, family=sm.families.Binomial(link=sm.families.links.log())).fit(disp = 0)
    
    params = model.params
    conf = np.exp(model.conf_int())
    conf['RR'] = np.exp(params)
    conf['z'] =model.tvalues
    conf['P>|z|'] =model.pvalues
    conf.columns = ['2.5%', '97.5%', 'RR', 'z', 'P>|z|']
    
    df = conf.loc[i:i]
    df['Feature'] = i
    df_directions_odd = pd.concat([df_directions_odd, df])

df_directions_odd = df_directions_odd.reset_index(drop=True)

df_directions_odd.to_excel('Results/long/long_RR_-with-slovakia.xlsx')
df_directions_odd

,2.5%,97.5%,RR,z,P>|z|,Feature
0,1.419603,1.518464,1.468202,22.361225,9.390011e-111,Mono
1,0.896125,0.979865,0.937060,-2.852483,4.337907e-03,One
2,0.734527,0.789933,0.761727,-14.670582,9.948823e-49,Two
3,0.630296,0.697404,0.663002,-15.922866,4.397616e-57,Three
4,0.657869,0.703752,0.680424,-22.386941,5.275830e-111,Total


### 1000-iteration

In [8]:


count = 0

results_dict = {}
for i in vars_:
    results_dict[i + "_y"] = pd.Series(dtype=float)
    results_dict[i + "_ypred"] = pd.Series(dtype=float)

results_merge_df_all = data.copy()


df_directions_odd = pd.DataFrame()

n_boosts = 10  
test_size = 500 / results_merge_df_all.shape[0]

for boosts in range(n_boosts):
    for i in vars_:

        y_ols = results_merge_df_all["GAP_bin"]
        X_ols = results_merge_df_all[[i] + ['delta_time']]

        X_train, X_test, y_train, y_test = train_test_split(
            X_ols, y_ols,
            test_size=test_size,
            stratify=results_merge_df_all["GAP_bin"],
            random_state=boosts
        )

        scaler = MinMaxScaler((0.05, 0.95))

        X_train_scaled = pd.DataFrame(
            scaler.fit_transform(X_train),
            columns=X_train.columns,
            index=X_train.index
        )
        X_test_scaled = pd.DataFrame(
            scaler.transform(X_test),
            columns=X_test.columns,
            index=X_test.index
        )

        X_train_scaled["intercept"] = 1
        X_test_scaled["intercept"] = 1


        model = sm.GLM(y_train, X_train_scaled, family=sm.families.Binomial(link=sm.families.links.log())).fit(disp = 0)


        y_test_pred = model.predict(X_test_scaled)

        results_dict[i + "_y"] = pd.concat([results_dict[i + "_y"], y_test], axis=0)
        results_dict[i + "_ypred"] = pd.concat([results_dict[i + "_ypred"], y_test_pred], axis=0)

        params = model.params
        conf = np.exp(model.conf_int())
        conf["RR"] = np.exp(params)
        conf["z"] = model.tvalues
        conf["P>|z|"] = model.pvalues
        conf.columns = ["2.5%", "97.5%", "RR", "z", "P>|z|"]

        df = conf.loc[i:i].copy()
        df["Feature"] = i

        hl10_stat, hl10_p = hosmer_lemeshow(y_test, y_test_pred, g=10)
        hl50_stat, hl50_p = hosmer_lemeshow(y_test, y_test_pred, g=50)
        hl100_stat, hl100_p = hosmer_lemeshow(y_test, y_test_pred, g=100)

        df["hl_statistic_10"] = hl10_stat
        df["hl_pvalue_10"] = hl10_p

        df["hl_statistic_50"] = hl50_stat
        df["hl_pvalue_50"] = hl50_p

        df["hl_statistic_100"] = hl100_stat
        df["hl_pvalue_100"] = hl100_p

        df_directions_odd = pd.concat([df_directions_odd, df], axis=0)

    df_directions_odd = df_directions_odd.reset_index(drop=True)

df_directions_odd;


In [9]:
df_directions_odd_f = pd.DataFrame()

for f in vars_:
    df_directions_odd_f = summarize_feature(df_directions_odd, f, df_directions_odd_f, stat_cols=(10, 50, 100))

df_directions_odd_f.to_excel('Results/long/long_RR_-with-slovakia-1000-iter.xlsx')
df_directions_odd_f

,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR
Mono,0.227057,0.276143,0.987386,1.458936,1.457314,1.460557,1.413560,1.507324
One,0.187174,0.301479,0.984956,0.931544,0.930323,0.932765,0.892904,0.971324
Two,0.124935,0.434685,0.987426,0.770669,0.770020,0.771317,0.745225,0.797220
Three,0.142060,0.342885,0.993240,0.668486,0.667222,0.669750,0.636319,0.701271
Total,0.233401,0.315816,0.988639,0.684755,0.683995,0.685515,0.662745,0.706768


## Odd ratios with macrosocials

### Without Iteration

In [10]:
vars_social_physical = ['gender_equal_l', 'Polution_conc_inv', 'Eq']

vars_sociopolitical = ['free_parties_l',  'inclu_suff_est', 'cred_elect_est', 'local_dem_est']

In [11]:
results_merge_df_all = data.copy()

covar_list = (
    ['total_exposomes', 'sociopolitical', 'social_physical', 'GDP',
     'number_leng_inst', 'number_stable_inst', 'distance', 'Migration']
    + vars_social_physical
    + vars_sociopolitical
)

df_directions_odd_all = []
df_directions_cov_odd_all = []

for covar in covar_list:
    for i in vars_:

        c_results_merge_df_all = results_merge_df_all.dropna(subset= [i, covar] + ['delta_time', 'GAP_bin'])
        c_results_merge_df_all.reset_index(drop = True, inplace = True)
        
        y_ols = c_results_merge_df_all['GAP_bin']
        X_ols = c_results_merge_df_all[[i, covar] + ['delta_time']]

        X_train, X_test, y_train, y_test = train_test_split(
            X_ols, y_ols, test_size=0.2, random_state=42
        )

        scaler = MinMaxScaler((0.05, 0.95))

        X_train_scaled = pd.DataFrame(
            scaler.fit_transform(X_train),
            columns=X_train.columns,
            index=X_train.index
        )
        X_test_scaled = pd.DataFrame(
            scaler.transform(X_test),
            columns=X_test.columns,
            index=X_test.index
        )

        X_train_scaled['intercept'] = 1
        X_test_scaled['intercept'] = 1

        model = sm.GLM(y_train, X_train_scaled, family=sm.families.Binomial(link=sm.families.links.log())).fit(disp = 0)

        params = model.params
        conf = np.exp(model.conf_int())
        conf['RR'] = np.exp(params)
        conf['z'] = model.tvalues
        conf['P>|z|'] = model.pvalues
        conf.columns = ['2.5%', '97.5%', 'RR', 'z', 'P>|z|']

        df_i = conf.loc[i:i].copy()
        df_i['Feature'] = i
        df_i['Covar'] = covar
        df_directions_odd_all.append(df_i)

        df_c = conf.loc[covar:covar].copy()
        df_c['Feature'] = f"{i}({covar})"
        df_c['Exposure'] = i          # opcional pero útil
        df_c['Covar'] = covar
        df_directions_cov_odd_all.append(df_c)

df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)
df_directions_cov_odd = pd.concat(df_directions_cov_odd_all, axis=0).reset_index(drop=True)

#df_directions_odd, df_directions_cov_odd


In [12]:
for covar in ['total_exposomes', 'sociopolitical', 'social_physical', 'GDP', 'number_leng_inst', 'number_stable_inst', 'distance', 'Migration'] + vars_social_physical + vars_sociopolitical:
    c_dfRR = df_directions_odd[df_directions_odd.Covar == covar]
    c_dfRR.to_excel('Results/long/long_RR_-with-slovakia_covar-' + covar + '.xlsx')

    print(covar)
    display(c_dfRR)

total_exposomes


,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
0,1.406743,1.507430,1.456217,21.312011,8.784104e-101,Mono,total_exposomes
1,0.901508,0.985567,0.942601,-2.599230,9.343327e-03,One,total_exposomes
2,0.745548,0.802873,0.773680,-13.578320,5.384712e-42,Two,total_exposomes
3,0.643152,0.712337,0.676861,-14.974136,1.083668e-50,Three,total_exposomes
4,0.662691,0.710191,0.686030,-21.338550,4.981539e-101,Total,total_exposomes


sociopolitical


,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
5,1.418875,1.517831,1.467519,22.302249,3.513908e-110,Mono,sociopolitical
6,0.892516,0.976205,0.933423,-3.013249,2.584669e-03,One,sociopolitical
7,0.734525,0.790259,0.761883,-14.576414,3.968198e-48,Two,sociopolitical
8,0.630547,0.697729,0.663288,-15.895658,6.791152e-57,Three,sociopolitical
9,0.658147,0.704117,0.680744,-22.328157,1.968788e-110,Total,sociopolitical


social_physical


...

,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
60,1.419131,1.518110,1.467786,22.311746,2.841866e-110,Mono,inclu_suff_est
61,0.895311,0.979142,0.936289,-2.883090,3.937946e-03,One,inclu_suff_est
62,0.735390,0.791128,0.762750,-14.530932,7.716380e-48,Two,inclu_suff_est
63,0.630395,0.697627,0.663159,-15.887992,7.674678e-57,Three,inclu_suff_est
64,0.658023,0.703986,0.680617,-22.337574,1.594736e-110,Total,inclu_suff_est


cred_elect_est


,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
65,1.407965,1.505233,1.455787,22.036949,1.274402e-107,Mono,cred_elect_est
66,0.904567,0.989389,0.946028,-2.426502,1.524517e-02,One,cred_elect_est
67,0.731895,0.786691,0.758799,-14.986215,9.035728e-51,Two,cred_elect_est
68,0.631447,0.698374,0.664068,-15.928853,3.996180e-57,Three,cred_elect_est
69,0.663669,0.709588,0.686244,-22.061322,7.437754e-108,Total,cred_elect_est


local_dem_est


,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
70,1.404179,1.502414,1.452466,21.637817,7.916344e-104,Mono,local_dem_est
71,0.908940,0.994475,0.950746,-2.201420,2.770634e-02,One,local_dem_est
72,0.738765,0.794458,0.766105,-14.369889,7.995932e-47,Two,local_dem_est
73,0.640155,0.707861,0.673157,-15.431221,1.009419e-53,Three,local_dem_est
74,0.664919,0.711507,0.687819,-21.661764,4.708540e-104,Total,local_dem_est


### 1000-Iteration

In [13]:

results_merge_df_all = data.copy()

covar_list = (
    ['total_exposomes', 'sociopolitical', 'social_physical', 'GDP',
     'number_leng_inst', 'number_stable_inst', 'distance', 'Migration']
    + vars_social_physical
    + vars_sociopolitical
)



n_boosts = 10  
test_size = 500 / results_merge_df_all.shape[0]


df_directions_odd_all = []
df_directions_cov_all = []


results_dict = {}  

for covar in covar_list:
    print(covar)
    for boosts in range(n_boosts):

        for i in vars_:

            c_results_merge_df_all = results_merge_df_all.dropna(subset= [i, covar] + ['delta_time', 'GAP_bin'])
            c_results_merge_df_all.reset_index(drop = True, inplace = True)
            
            y_ols = c_results_merge_df_all['GAP_bin']
            X_ols = c_results_merge_df_all[[i, covar] + ['delta_time']]

            # Split igual al primero: test fijo ~500 y estratificado
            X_train, X_test, y_train, y_test = train_test_split(
                X_ols, y_ols,
                test_size=test_size,
                stratify=y_ols,
                random_state=boosts
            )

            scaler = MinMaxScaler((0.05, 0.95))

            X_train_scaled = pd.DataFrame(
                scaler.fit_transform(X_train),
                columns=X_train.columns,
                index=X_train.index
            )

            X_test_scaled = pd.DataFrame(
                scaler.transform(X_test),
                columns=X_test.columns,
                index=X_test.index
            )


            X_train_scaled["intercept"] = 1
            X_test_scaled["intercept"] = 1


            model = sm.GLM(y_train, X_train_scaled, family=sm.families.Binomial(link=sm.families.links.log())).fit(disp = 0)
            y_test_pred = model.predict(X_test_scaled)

            key_y = (covar, i, "y")
            key_p = (covar, i, "ypred")
            if key_y not in results_dict:
                results_dict[key_y] = pd.Series(dtype=float)
                results_dict[key_p] = pd.Series(dtype=float)

            results_dict[key_y] = pd.concat([results_dict[key_y], y_test], axis=0)
            results_dict[key_p] = pd.concat([results_dict[key_p], y_test_pred], axis=0)

            params = model.params
            conf = np.exp(model.conf_int())
            conf["RR"] = np.exp(params)
            conf["z"] = model.tvalues
            conf["P>|z|"] = model.pvalues
            conf.columns = ["2.5%", "97.5%", "RR", "z", "P>|z|"]

            hl10_stat, hl10_p = hosmer_lemeshow(y_test, y_test_pred, g=10)
            hl50_stat, hl50_p = hosmer_lemeshow(y_test, y_test_pred, g=50)
            hl100_stat, hl100_p = hosmer_lemeshow(y_test, y_test_pred, g=100)

            df_i = conf.loc[i:i].copy()
            df_i["Feature"] = i
            df_i["Covar"] = covar
            df_i["boosts"] = boosts

            df_i["hl_statistic_10"] = hl10_stat
            df_i["hl_pvalue_10"] = hl10_p
            df_i["hl_statistic_50"] = hl50_stat
            df_i["hl_pvalue_50"] = hl50_p
            df_i["hl_statistic_100"] = hl100_stat
            df_i["hl_pvalue_100"] = hl100_p

            df_directions_odd_all.append(df_i)

            df_c = conf.loc[covar:covar].copy()
            df_c["Feature"] = i + "(" + covar + ")"
            df_c["Exposure"] = i
            df_c["Covar"] = covar
            df_c["boosts"] = boosts

            df_c["hl_statistic_10"] = hl10_stat
            df_c["hl_pvalue_10"] = hl10_p
            df_c["hl_statistic_50"] = hl50_stat
            df_c["hl_pvalue_50"] = hl50_p
            df_c["hl_statistic_100"] = hl100_stat
            df_c["hl_pvalue_100"] = hl100_p

            df_directions_cov_all.append(df_c)

df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)
df_directions_cov_odd = pd.concat(df_directions_cov_all, axis=0).reset_index(drop=True)

#df_directions_odd, df_directions_cov_odd


total_exposomes
sociopolitical
social_physical
GDP
number_leng_inst
number_stable_inst
distance
Migration
gender_equal_l
Polution_conc_inv
Eq
free_parties_l
inclu_suff_est
cred_elect_est
local_dem_est


In [14]:

features = list(vars_) 
covars = df_directions_odd["Covar"].dropna().unique()

idx = pd.MultiIndex.from_product([covars, features], names=["Covar", "Feature"])
df_directions_odd_f = pd.DataFrame(index=idx)

for c in covars:
    for f in features:
        df_directions_odd_f = summarize_feature_by_covar(
            df_directions_odd, c, f, df_directions_odd_f,
            stat_cols=(10, 50, 100),
            df_offset=2
        )

df_directions_odd_f = df_directions_odd_f.reset_index()
df_directions_odd_f;


In [15]:
for covar in ['total_exposomes', 'sociopolitical', 'social_physical', 'GDP', 'number_leng_inst', 'number_stable_inst', 'distance', 'Migration'] + vars_social_physical + vars_sociopolitical:
    c_dfRR = df_directions_odd_f[df_directions_odd_f.Covar == covar]
    c_dfRR.to_excel('Results/long/long_RR_-with-slovakia_covar-' + covar + '-1000-iter.xlsx')

    print(covar)
    display(c_dfRR)

total_exposomes


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
0,total_exposomes,Mono,0.237175,0.247340,0.912634,1.450242,1.448545,1.451940,1.404087,1.500047,10.0
1,total_exposomes,One,0.358734,0.329769,0.919471,0.936901,0.935695,0.938108,0.898114,0.976825,10.0
2,total_exposomes,Two,0.116258,0.280230,0.898125,0.781196,0.780519,0.781872,0.754871,0.808682,10.0
3,total_exposomes,Three,0.200831,0.330472,0.913018,0.680492,0.679160,0.681823,0.647220,0.714152,10.0
4,total_exposomes,Total,0.241600,0.256812,0.907312,0.688865,0.688061,0.689670,0.665966,0.711541,10.0


sociopolitical


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
5,sociopolitical,Mono,0.221977,0.315780,0.984102,1.457372,1.455779,1.458966,1.411979,1.505596,10.0
6,sociopolitical,One,0.243152,0.426514,0.987146,0.926626,0.925388,0.927864,0.888042,0.966408,10.0
7,sociopolitical,Two,0.118466,0.333425,0.987343,0.771730,0.771085,0.772375,0.746059,0.798455,10.0
8,sociopolitical,Three,0.169853,0.476426,0.992384,0.668985,0.667728,0.670243,0.636807,0.701829,10.0
9,sociopolitical,Total,0.187772,0.294545,0.983633,0.685491,0.684742,0.686239,0.663507,0.707559,10.0


social_physical


...

,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
60,inclu_suff_est,Mono,0.262051,0.358217,0.995124,1.458357,1.456737,1.459978,1.412919,1.506819,10.0
61,inclu_suff_est,One,0.265051,0.303946,0.984958,0.930482,0.929252,0.931712,0.891762,0.970318,10.0
62,inclu_suff_est,Two,0.137531,0.363845,0.991162,0.772249,0.771607,0.772892,0.746583,0.798937,10.0
63,inclu_suff_est,Three,0.214036,0.377154,0.989280,0.668708,0.667441,0.669975,0.636482,0.701591,10.0
64,inclu_suff_est,Total,0.262001,0.377784,0.995351,0.685029,0.684269,0.685789,0.662970,0.707090,10.0


cred_elect_est


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
65,cred_elect_est,Mono,0.322074,0.361601,0.989529,1.449109,1.447435,1.450782,1.404254,1.497190,10.0
66,cred_elect_est,One,0.350082,0.309806,0.978234,0.939301,0.938112,0.940490,0.900279,0.979609,10.0
67,cred_elect_est,Two,0.306262,0.448294,0.988601,0.767919,0.767300,0.768537,0.742689,0.794141,10.0
68,cred_elect_est,Three,0.253701,0.403194,0.984289,0.669174,0.667915,0.670432,0.637078,0.701850,10.0
69,cred_elect_est,Total,0.322208,0.356345,0.990791,0.689413,0.688617,0.690208,0.667243,0.711466,10.0


local_dem_est


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
70,local_dem_est,Mono,0.224671,0.387272,0.987610,1.445559,1.443881,1.447236,1.400347,1.493977,10.0
71,local_dem_est,One,0.183912,0.420966,0.989126,0.943992,0.942765,0.945219,0.904618,0.984568,10.0
72,local_dem_est,Two,0.173238,0.405058,0.991013,0.774558,0.773903,0.775213,0.749012,0.801206,10.0
73,local_dem_est,Three,0.302593,0.453007,0.986694,0.677223,0.675917,0.678529,0.644678,0.710267,10.0
74,local_dem_est,Total,0.230935,0.389626,0.988762,0.691110,0.690309,0.691911,0.668683,0.713456,10.0


In [16]:
dt = time.perf_counter() - t0
m, s = divmod(dt, 60)
print(f"Tiempo total: {int(m)} min {s:.1f} s")

Tiempo total: 5 min 6.1 s
